In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os  

In [4]:
os.getcwd() 

'/root/data/experiments/exp_birds2'

In [5]:
import sys                                                                             # Python system library needed to load custom functions
import numpy as np                                                                     # for performing calculations on numerical arrays
import pandas as pd                                                                    # home of the DataFrame construct, _the_ most important object for Data Science
import seaborn as sns                                                                  # additional plotting library
import matplotlib.pyplot as plt                                                        # allows creation of insightful plots
import os                                                                              # for changing the directory

import sagemaker                                                                       # dedicated sagemaker library to execute training jobs
import boto3                                                                           # for interacting with S3 buckets

from sagemaker.huggingface import HuggingFace                                           # for executing the trainig jobs
from sklearn.metrics import precision_recall_fscore_support, accuracy_score             # tools to understand how our model is performing

#sys.path.append('')                                                               # Add the source directory to the PYTHONPATH. This allows to import local functions and modules.
from config import DEFAULT_BUCKET, DEFAULT_REGION  
from gdsc_utils import create_encrypted_bucket, download_and_extract_model, PROJECT_DIR # functions to create S3 buckets and to help with downloading models. Importing our root directory
from gdsc_eval import plot_confusion_matrix                                             # function for creating confusion matrix                                     # importing the bucket name that contains data for the challenge and the default region
os.chdir(PROJECT_DIR)                                                                   # changing our directory to root


import logging                                                    # module for displaying relevant information in the logs
import sys                                                        # to access to some variables used or maintained by the interpreter 
import argparse                                                   # to parse arguments from passed in the hyperparameters
import json                                                       # to open the json file with labels
from transformers import (                                        # required classes to perform the model training and implement early stopping
    ASTFeatureExtractor, 
    ASTForAudioClassification, 
    Trainer, 
    TrainingArguments, 
    EarlyStoppingCallback
)                                    
import torch                                                       # library to work with PyTorch tensors and to figure out if we have a GPU available
from datasets import load_dataset, Audio, Dataset                  # required tools to create, load and process our audio dataset
from preprocessing import preprocess_audio_arrays                  # functions to preprocess the dataset with ASTFeatureExtractor
from gdsc_eval import compute_metrics, make_predictions            # functions to create predictions and evaluate them
from typing import Optional                                        # for type hints

from collections import Counter
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
def get_feature_extractor(model_name: str, 
                          train_dataset_mean: Optional[float] = None, 
                          train_dataset_std: Optional[float] = None) -> ASTFeatureExtractor:
    """
    Retrieves a feature extractor for audio signal processing.

    Args:
        model_name (str): The name of the pre-trained model to use.
        train_dataset_mean (float, optional): The mean value of the training dataset. Defaults to None.
        train_dataset_std (float, optional): The standard deviation of the training dataset. Defaults to None.

    Returns:
        ASTFeatureExtractor: An instance of the ASTFeatureExtractor class.

    """
    if all((train_dataset_mean, train_dataset_std)):
        feature_extractor = ASTFeatureExtractor.from_pretrained(model_name, mean=train_dataset_mean, std=train_dataset_std, max_length=1024)
        logger.info(f" feature extractor loaded with dataset mean: {train_dataset_mean} and standard deviation: {train_dataset_std}")
    else:
        feature_extractor = ASTFeatureExtractor.from_pretrained(model_name)
        logger.info(" at least one of the optional arguments (mean, std) is missing")
        logger.info(f" feature extractor loaded with default dataset mean: {feature_extractor.mean} and standard deviation: {feature_extractor.std}")
        
    return feature_extractor

def preprocess_data_for_training(
    dataset_path: str,
    sampling_rate: int,
    feature_extractor: ASTFeatureExtractor,
    fe_batch_size: int,
    dataset_name: str,
    shuffle: bool = False,
    extract_file_name: bool = True) -> Dataset:
    """
    Preprocesses audio data for training.

    Args:
        dataset_path (str): The path to the dataset.
        sampling_rate (int): The desired sampling rate for the audio.
        feature_extractor (ASTFeatureExtractor): The feature extractor to use for preprocessing.
        fe_batch_size (int): The batch size for feature extraction.
        dataset_name (str, optional): The name of the dataset. Defaults to None.
        shuffle (bool, optional): Whether to shuffle the dataset. Defaults to False.
        extract_file_name (bool, optional): Whether to extract paths from audio features. Defaults to True.

    Returns:
        dataset: The preprocessed dataset.

    """
    dataset = load_dataset("audiofolder", data_dir=dataset_path).get('train') # loading the dataset
    
    # perform shuffle if specified
    if shuffle:
        dataset = dataset.shuffle(seed=42)
        
    logger.info(f" loaded {dataset_name} dataset length is: {len(dataset)}")

    if extract_file_name:
        remove_metadata = lambda x: x.endswith(".wav")
        extract_file_name = lambda x: x.split('/')[-1]

        dataset_paths = list(dataset.info.download_checksums.keys())
        dataset_paths = list(filter(remove_metadata, dataset_paths))
        dataset_paths = list(map(extract_file_name, dataset_paths))
        dataset = dataset.add_column("file_name", dataset_paths)

    dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))
    
    logger.info(f" {dataset_name} dataset sampling rate casted to: {sampling_rate}")

    dataset_encoded = dataset.map(
        lambda x: preprocess_audio_arrays(x, 'audio', 'array', feature_extractor),
        remove_columns="audio",
        batched=True,
        batch_size=fe_batch_size
    )
    
    logger.info(f" done extracting features for {dataset_name} dataset")
    
    return dataset_encoded

In [7]:
parser = argparse.ArgumentParser()

# hyperparameters sent from our jupyter notebook are passed as command-line arguments to the script
# preprocessing hyperparameters
parser.add_argument("--sampling_rate", type=int, default=22050)                        # sampling rate to which we will cast audio files
parser.add_argument("--fe_batch_size", type=int, default=32)                           # feature extractor batch size
parser.add_argument("--train_dataset_mean", type=float, default=-8.076275929131292)                  # mean value of spectrograms of our data
parser.add_argument("--train_dataset_std", type=float, default=3.984092920341275)                   # standard deviation value of spectrograms of our resampled data
# parser.add_argument("--train_dataset_mean", type=float, default=-8.141991150530815)                  # mean value of spectrograms of our data
# parser.add_argument("--train_dataset_std", type=float, default=4.095692486358449)                   # standard deviation value of spectrograms of our resampled data

# training hyperparameters
parser.add_argument("--model_name", type=str)                                          # name of the pretrained model from HuggingFace
parser.add_argument("--learning_rate", type=float, default=2e-5)                       # learning rate
parser.add_argument("--epochs", type=int, default=4)                                   # number of training epochs 
parser.add_argument("--train_batch_size", type=int, default=4)                        # training batch size
parser.add_argument("--eval_batch_size", type=int, default=64)                         # evaluation batch size
parser.add_argument("--patience", type=int, default=2)                                 # early stopping - how many epoch without improvement will stop the training 
# parser.add_argument("--data_channel", type=str, default=os.environ["SM_CHANNEL_DATA"]) # directory where input data from S3 is stored
parser.add_argument("--train_dir", type=str, default="train")                          # folder name with training data
parser.add_argument("--val_dir", type=str, default="val")                              # folder name with validation data
parser.add_argument("--test_dir", type=str, default="test")                            # folder name with test data
# parser.add_argument("--output_dir", type=str, default=os.environ['SM_MODEL_DIR'])      # output directory. This directory will be saved in the S3 bucket


args, _ = parser.parse_known_args()                    # parsing arguments from the notebook


# train_path = f"{args.data_channel}/{args.train_dir}"   # directory of our training dataset on the instance
# val_path = f"{args.data_channel}/{args.val_dir}"       # directory of our validation dataset on the instance
# test_path = f"{args.data_channel}/{args.test_dir}"     # directory of our test dataset on the instance

# Set up logging which allows to print information in logs
logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.getLevelName("INFO"),
    handlers=[logging.StreamHandler(sys.stdout)],
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

In [8]:
train_path = 'data/data_small/train'
val_path = 'data/data_small/val'

In [9]:
with open(f'data/data_small/labels.json', 'r') as f:
        labels = json.load(f)
    
# Create mapping from label to id and id to label
label2id, id2label = dict(), dict()
for k, v in labels.items():
    label2id[k] = str(v)
    id2label[str(v)] = k

num_labels = len(label2id)  # define number of labels
output_dir='models/efficientnet2v4',                # directory for saving model checkpoints and logs
args.model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"

In [10]:
# feature_extractor = get_feature_extractor(args.model_name, args.train_dataset_mean, args.train_dataset_std)
feature_extractor = ASTFeatureExtractor.from_pretrained(args.model_name, do_normalize=False)

# creating train and validation datasets
train_dataset_encoded = preprocess_data_for_training(dataset_path=train_path, sampling_rate=args.sampling_rate, feature_extractor=feature_extractor,
                                                     fe_batch_size=args.fe_batch_size, dataset_name="train", shuffle=True, extract_file_name=False)

val_dataset_encoded = preprocess_data_for_training(dataset_path=val_path, sampling_rate=args.sampling_rate, feature_extractor=feature_extractor,
                                                   fe_batch_size=args.fe_batch_size, dataset_name="validation")

# Download model from model hub
model = ASTForAudioClassification.from_pretrained(args.model_name, num_labels=num_labels, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)


Resolving data files:   0%|          | 0/177 [00:00<?, ?it/s]

2023-07-10 20:31:15,492 - datasets.builder - WARNING - Found cached dataset audiofolder (/root/.cache/huggingface/datasets/audiofolder/default-4ea904e8a4f39112/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)


  0%|          | 0/1 [00:00<?, ?it/s]

2023-07-10 20:31:15,508 - datasets.arrow_dataset - WARNING - Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/audiofolder/default-4ea904e8a4f39112/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc/cache-8d133f78b720c65d.arrow
2023-07-10 20:31:15,511 - __main__ - INFO -  loaded train dataset length is: 176
2023-07-10 20:31:15,515 - __main__ - INFO -  train dataset sampling rate casted to: 22050
2023-07-10 20:31:15,529 - datasets.arrow_dataset - WARNING - Loading cached processed dataset at /root/.cache/huggingface/datasets/audiofolder/default-4ea904e8a4f39112/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc/cache-2cdba842eef158bb.arrow
2023-07-10 20:31:15,530 - __main__ - INFO -  done extracting features for train dataset


Resolving data files:   0%|          | 0/67 [00:00<?, ?it/s]

2023-07-10 20:31:15,683 - datasets.builder - WARNING - Found cached dataset audiofolder (/root/.cache/huggingface/datasets/audiofolder/default-99123276a0f5ab0b/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)


  0%|          | 0/1 [00:00<?, ?it/s]

2023-07-10 20:31:15,699 - __main__ - INFO -  loaded validation dataset length is: 66
2023-07-10 20:31:15,715 - __main__ - INFO -  validation dataset sampling rate casted to: 22050
2023-07-10 20:31:15,726 - datasets.arrow_dataset - WARNING - Loading cached processed dataset at /root/.cache/huggingface/datasets/audiofolder/default-99123276a0f5ab0b/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc/cache-96be9f0276ba55fe.arrow
2023-07-10 20:31:15,726 - __main__ - INFO -  done extracting features for validation dataset


Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([66, 768]) in the model instantiated
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([66]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
train_dataset_encoded

Dataset({
    features: ['label', 'input_values'],
    num_rows: 176
})

In [12]:
input_train_data = np.array([train_dataset_encoded[i]['input_values'] for i in range(len(train_dataset_encoded))])

In [13]:
input_train_data = abs(input_train_data)

In [14]:
input_train_data = np.repeat(input_train_data[..., np.newaxis], 3, axis=-1)

In [15]:
input_train_data.shape

(176, 1024, 128, 3)

In [16]:
from transformers import AutoImageProcessor, EfficientNetForImageClassification

In [17]:
image_processor = AutoImageProcessor.from_pretrained("google/efficientnet-b4")

In [18]:
x_norm = (input_train_data-np.min(input_train_data))/(np.max(input_train_data)-np.min(input_train_data))

In [19]:
inputs = image_processor(x_norm, return_tensors="pt")

In [37]:
example_input = inputs['pixel_values'][0]

In [38]:
example_input = example_input[None, :, :, :]

In [39]:
dct = {'pixel_values' : example_input}

In [40]:
dct

{'pixel_values': tensor([[[[ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
           [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
           [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
           ...,
           [ 2.2489,  2.2489,  2.2489,  ..., -0.6965, -0.6965, -0.6965],
           [ 1.3755,  1.3755,  1.3755,  ..., -1.9980, -1.9980, -1.9980],
           [ 2.2489,  2.2489,  2.2489,  ..., -0.7137, -0.7137, -0.7137]],
 
          [[ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
           [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
           [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
           ...,
           [ 2.4286,  2.4286,  2.4286,  ..., -0.5826, -0.5826, -0.5826],
           [ 1.5357,  1.5357,  1.5357,  ..., -1.9132, -1.9132, -1.9132],
           [ 2.4286,  2.4286,  2.4286,  ..., -0.6001, -0.6001, -0.6001]],
 
          [[ 2.6400,  2.6400,  2.6400,  ...,  2.6400,  2.6400,  2.6400

In [41]:
example_input.shape

torch.Size([1, 3, 380, 380])

In [42]:
model = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b4", num_labels=66, ignore_mismatched_sizes=True)

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b4 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1792]) in the checkpoint and torch.Size([66, 1792]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([66]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
labels = []
for i in tqdm(range(176)):
    example_input = inputs['pixel_values'][i]
    example_input = example_input[None, :, :, :]
    dct = {'pixel_values' : example_input}
    with torch.no_grad():
        logits = model(**dct).logits
        predicted_label = logits.argmax(-1).item()
        labels.append(predicted_label)

100%|██████████| 176/176 [00:33<00:00,  5.33it/s]


In [44]:
logits

tensor([[0.0156, 0.0150, 0.0140, 0.0159, 0.0196, 0.0144, 0.0108, 0.0139, 0.0159,
         0.0151, 0.0147, 0.0174, 0.0188, 0.0174, 0.0176, 0.0161, 0.0122, 0.0153,
         0.0159, 0.0157, 0.0132, 0.0129, 0.0195, 0.0130, 0.0150, 0.0155, 0.0128,
         0.0163, 0.0145, 0.0144, 0.0155, 0.0186, 0.0109, 0.0176, 0.0152, 0.0151,
         0.0177, 0.0141, 0.0163, 0.0131, 0.0194, 0.0140, 0.0175, 0.0091, 0.0166,
         0.0145, 0.0140, 0.0134, 0.0115, 0.0122, 0.0164, 0.0158, 0.0110, 0.0175,
         0.0163, 0.0134, 0.0123, 0.0136, 0.0172, 0.0186, 0.0145, 0.0140, 0.0163,
         0.0163, 0.0156, 0.0166]])

In [45]:
predicted_label = logits.argmax(-1).item()

In [46]:
predicted_label

4

In [49]:
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.rnd = np.random.RandomState(0)

        #self.x_data = torch.tensor(train_dataset_encoded[:]['input_values'], dtype=torch.float32).to(device)
        self.x_data = np.array([train_dataset_encoded[i]['input_values'] for i in range(len(train_dataset_encoded))])
        self.x_data = abs(self.x_data)
        self.x_data = np.repeat(self.x_data[..., np.newaxis], 3, axis=-1)
        self.x_data = (self.x_data-np.min(self.x_data))/(np.max(self.x_data)-np.min(self.x_data))
        self.y_data = torch.tensor(train_dataset_encoded[:]['label'], dtype=torch.int64).to(device) 

        self.n = len(self.x_data)

    def __len__(self):
        return self.n

#TODO: create better triplet selection
    def __getitem__(self, idx):
        y = self.y_data[idx]
        x = self.x_data[idx]
        #x = x.resize_(1, 1024, 128)
        #x = x.expand(3, 1024, 128)
        return (x, y)

In [50]:
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification

class ClassificationNet(torch.nn.Module):
    def __init__(self):
        super(ClassificationNet, self).__init__()  # pre 3.3 syntax
        self.model = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b4", num_labels=66, ignore_mismatched_sizes=True)
        self.image_processor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b4")
        
    def forward(self, x):
        inputs = self.image_processor(x, return_tensors="pt")
        output = self.model(x).logits
        return output

### TRAIN SIAMESE PART

In [51]:
def make_predictions2(examples: torch.Tensor, 
                     model: torch.nn.Module, 
                     device,
                     labels: torch.Tensor = None):
    model = model.to(device)
    examples = torch.tensor(examples, dtype=torch.float32).to(device)
    examples = examples.resize_(examples.shape[0], 1,examples.shape[-2],examples.shape[-1])
    examples = examples.expand(examples.shape[0], 3,examples.shape[-2],examples.shape[-1])
    labels = torch.tensor(labels, dtype=torch.int64).to(device) 

    with torch.no_grad():
        logits = model(examples)
    predicted_class_id = [str(torch.argmax(item).item()) for item in logits]
    if isinstance(labels, torch.Tensor):
        loss = torch.nn.functional.cross_entropy(logits.view(-1, 66), labels.to(device).view(-1), reduction="none")
        loss = loss.view(len(examples), -1).cpu().numpy()

        return {'predicted_class_id': predicted_class_id, 'loss': loss, 'logits': logits}
    else:
        return {'predicted_class_id': predicted_class_id}

    
def compute_metrics(pred, labels):
    # labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average="macro")
    acc = accuracy_score(labels, pred)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

### TRAIN CLASSIFICATION PART

In [52]:
lrn_rate = 1e-3
max_epochs = 10
ep_log_interval = 1
batch_size_class = 8

train_class_ds = ClassificationDataset()
train_class_ldr = torch.utils.data.DataLoader(train_class_ds, batch_size=batch_size_class, shuffle=True)

classification_net = ClassificationNet().to(device) #pass net.model - siamese model
optimizer_class = torch.optim.Adam(classification_net.parameters(), lr=lrn_rate)
categorical_crossentropy = torch.nn.functional.cross_entropy

for epoch in range(0, max_epochs):
    ep_loss = 0  # for one full epoch
    for (batch_idx, batch) in tqdm(enumerate(train_class_ldr)):
        X, y = batch
        output = classification_net(X)

        optimizer_class.zero_grad()       # reset gradients
        loss_val = categorical_crossentropy(output, y)

        ep_loss += loss_val.item()  # accumulate loss
        loss_val.backward()         # compute grads
        optimizer_class.step()            # update weights
    if epoch % ep_log_interval == 0:
        print("epoch = %4d  |  loss = %10.4f" % (epoch, ep_loss))
        val_dataset_encoded2 = val_dataset_encoded.map(lambda x: make_predictions2(x['input_values'], classification_net, device, x['label']), batched=True, batch_size=8, remove_columns="input_values")
        print(compute_metrics([int(i) for i in val_dataset_encoded2[:]['predicted_class_id']], val_dataset_encoded2[:]['label']))
    print("Done ") 

Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b4 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1792]) in the checkpoint and torch.Size([66, 1792]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([66]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
0it [00:00, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_313/810896214.py:17 in <module>                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_313/810896214.py'                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1194 in _call_impl            │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /tmp/ipykernel_313/2533542190.py:11 in forward                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_313/2533542190.py'                          │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1194 in _call_impl            │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/models/efficientnet/modeling_efficientnet.p │
│ y:618 in forward                                                                                 │
│                                                                                                  │
│   615 │   │   """                                                                                │
│   616 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return   │
│   617 │   │                                                                                      │
│ ❱ 618 │   │   outputs = self.efficientnet(pixel_values, output_hidden_states=output_hidden_sta   │
│   619 │   │                                                                                      │
│   620 │   │   pooled_output = outputs.pooler_output if return_dict else outputs[1]               │
│   621 │   │   pooled_output = self.dropout(pooled_output)                                        │
│                                                            

In [22]:
val_dataset_encoded2 = val_dataset_encoded.map(lambda x: make_predictions2(x['input_values'], classification_net, device, x['label']), batched=True, batch_size=4, remove_columns="input_values")

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

In [26]:
compute_metrics([int(i) for i in val_dataset_encoded2[:]['predicted_class_id']], val_dataset_encoded2[:]['label'])

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 0.5909090909090909,
 'f1': 0.4952380952380952,
 'precision': 0.4532828282828283,
 'recall': 0.5909090909090909}

In [51]:
X

tensor([[[[-15.9424, -15.9424, -15.2467,  ...,  -9.9298, -10.5248, -14.6571],
          [-15.9424, -15.9424, -15.9424,  ...,  -9.9403, -11.0747, -14.0391],
          [-15.9424, -15.9424, -15.3227,  ...,  -8.6226, -10.2443, -15.4034],
          ...,
          [-15.9424, -15.9424, -15.9424,  ...,  -9.4018, -11.1095, -13.0079],
          [-15.9424, -15.9424, -15.5424,  ...,  -9.5473, -11.4716, -13.6956],
          [-15.8215, -15.9424, -15.4332,  ...,  -9.7442, -11.2408, -15.4254]],

         [[-15.9424, -15.9424, -15.2467,  ...,  -9.9298, -10.5248, -14.6571],
          [-15.9424, -15.9424, -15.9424,  ...,  -9.9403, -11.0747, -14.0391],
          [-15.9424, -15.9424, -15.3227,  ...,  -8.6226, -10.2443, -15.4034],
          ...,
          [-15.9424, -15.9424, -15.9424,  ...,  -9.4018, -11.1095, -13.0079],
          [-15.9424, -15.9424, -15.5424,  ...,  -9.5473, -11.4716, -13.6956],
          [-15.8215, -15.9424, -15.4332,  ...,  -9.7442, -11.2408, -15.4254]],

         [[-15.9424, -15.9424,